# Step 1: Scraping MTA subway alerts 
### data source: *[MTA Alert Archive](https://mymtaalerts.com/archive)*

In [87]:
# Opening up the page using playwright
from playwright.async_api import async_playwright

In [88]:
url = 'https://mymtaalerts.com/archive'

In [89]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [90]:
# Go to NYCT subway's archive page
await page.goto(url)

<Response url='https://mymtaalerts.com/archive' request=<Request url='https://mymtaalerts.com/archive' method='GET'>>

### Narrow down search to November 

In [91]:
# click on "hide elevator & escalator alerts"
await page.locator("#ctl00_ContentPlaceHolder1_chkHideElevatorEscalator").click()

In [92]:
# select alerts from NYCT Subway only
await page.locator("#ctl00_ContentPlaceHolder1_ddlAgency").select_option('NYC')

['NYC']

In [93]:
# select date from 11.1 to 11.30
await page.locator("#ctl00_ContentPlaceHolder1_dtpStartDate_dateInput").fill('11/01/2022')
await page.locator("#ctl00_ContentPlaceHolder1_dtpStopDate_dateInput").fill('11/30/2022')

In [94]:
# Click on "update" button
await page.locator("#ctl00_ContentPlaceHolder1_btnGetData").click()

### Look at the table on the first page

In [95]:
import pandas as pd

In [1]:
html = await page.content()
tables = pd.read_html(html)
df_p1 = tables[8]
df_p1

pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

df_p1.columns = ['date','agency','subject','message']
df_p1

NameError: name 'page' is not defined

In [101]:
# await page.locator("xpath=/html/body/div[1]/div/div/div/form/div[7]/table/thead/tr[2]/td/div/div[3]/button[1]").click()
# 'button:text("Button Text Here")'

current_page = await page.query_selector(".rgCurrentPage")
this_page = await current_page.text_content()


'61'

In [129]:
page_info = await page.query_selector("xpath=/html/body/div[1]/div/div/div/form/div[7]/table/thead/tr[2]/td/div/div[5]/strong[2]")
await page_info.text_content()

'61'

### Start scraping!

In [96]:
# await page.locator('.t-button rgActionButton rgPageNext').click()
import time


In [146]:
# scrape the number of pages and compare with, if 61 == 61, break
dataframes = []
while True:
    time.sleep(3)
    current_page = await page.query_selector("xpath=/html/body/div[1]/div/div/div/form/div[7]/table/thead/tr[2]/td/div/div[2]/a[11]")
    this_page = await current_page.text_content()

    # Grabbing the number of current page and the number of last page
    page_info = await page.query_selector("xpath=/html/body/div[1]/div/div/div/form/div[7]/table/thead/tr[2]/td/div/div[5]/strong[2]")
    last_page = await page_info.text_content()

    # If the current number and the last page number does not match,
    if this_page != last_page:
        # scrape the current page
        content = await page.content()
        tables = pd.read_html(content)
        # read in the alert table
        print("scraped a page")
        df = tables[8]
        # rename columns for consistency
        df.columns = ['date','agency','subject','message']
        dataframes.append(df)
        # flip to the next page
        await page.locator("xpath=/html/body/div[1]/div/div/div/form/div[7]/table/thead/tr[2]/td/div/div[3]/button[1]").click()
        print("click the button")
        
    # If the current page number matches the number of last page 
    elif this_page == last_page:
        # scraped the page, read in the table
        content = await page.content()
        tables = pd.read_html(content)
        print("scraped a page")
        df = tables[8]
        df.columns = ['date','agency','subject','message']
        dataframes.append(df)
        print("got to the last page")
        # Using break to escape the loop
        break

scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped a page
click the button
scraped 

In [147]:
# Compile all tables into a giant dataframe
df = pd.concat(dataframes, ignore_index = True)
df.shape

(3147, 4)

In [148]:
df.head()

,date,agency,subject,message
3142,11/1/22 12:38 AM,NYC,"Update: BKLYN, F Train, Delays",Southbound F trains are running with delays after we addressed a signal problem near Church Av.
3143,11/1/22 12:19 AM,NYC,"Update: BKLYN, D, N, and R Trains, Delays",Southbound D/N/R trains are running with delays after NYPD responded to a person being disruptive aboard a train at DeKalb Av.
3144,11/1/22 12:00 AM,NYC,"BKLYN, D, N, and R Trains, Delays",Southbound D/N/R trains are delayed while we request NYPD assistance for a person being disruptive aboard a train at DeKalb Av.
3145,Menu,Menu,Menu,Menu
3146,...52535455565758596061 Page size: select 3025 items in 61 pages,...52535455565758596061 Page size: select 3025 items in 61 pages,...52535455565758596061 Page size: select 3025 items in 61 pages,...52535455565758596061 Page size: select 3025 items in 61 pages


### Save as .csv file

In [2]:
# import dataframe as csv
df.to_csv('november-delays.csv', index = False)

NameError: name 'df' is not defined

For cleaning process, please see notebook *```Step 02 - Cleaning```*.